In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RBFInterpolator


In [8]:
path="/home/jorgebdelafuente/Doctorado/PH2M/Sings/"
dm_entry_11 = np.loadtxt(path+"entry/dm_1.1.dat")
dm_entry_12 = np.loadtxt(path+"entry/dm_1.2.dat")
dm_exit_11 = np.loadtxt(path+"exit/dm_1.1.dat")
dm_exit_12 = np.loadtxt(path+"exit/dm_1.2.dat")

In [9]:
def get_ic(rg,rp,theta):
    thrad=np.radians(theta)
    r12=((rp*0.50)*(rp*0.50)+rg*rg-2.0*0.50*rp*rg*np.cos(thrad))**0.50
    r13=((rp*0.50)*(rp*0.50)+rg*rg-2.0*0.50*rp*rg*np.cos(np.pi-thrad))**0.50
    r23=rp
    return r12,r13,r23

In [10]:
dm_11_entry_ic = np.zeros((len(dm_entry_11), 6))
dm_12_entry_ic = np.zeros((len(dm_entry_12), 6))

for i in range(len(dm_entry_11)):
    r12, r13, r23 = get_ic(dm_entry_11[i, 0], dm_entry_11[i, 1], dm_entry_11[i, 1])
    # coge las columnas 3,4,5 (índices 3:6)
    dm_11_entry_ic[i, :] = np.array([r12, r13, r23,
                                    dm_entry_11[i, 3], dm_entry_11[i, 4], dm_entry_11[i, 5]])

for i in range(len(dm_entry_12)):
    r12, r13, r23 = get_ic(dm_entry_12[i, 0], dm_entry_12[i, 1], dm_entry_12[i, 2])
    dm_12_entry_ic[i, :] = np.array([r12, r13, r23,
                                    dm_entry_12[i, 3], dm_entry_12[i, 4], dm_entry_12[i, 5]])


In [11]:
dm_mod_11 = np.zeros((len(dm_11_entry_ic)+len(dm_exit_11), 4))
dm_mod_12 = np.zeros((len(dm_12_entry_ic)+len(dm_exit_12), 4))

def fill_mod(out, first, second):
    n1 = len(first)
    out[:n1, :3] = first[:, :3]
    out[:n1,  3] = np.linalg.norm(first[:, 3:6], axis=1)
    out[n1:, :3] = second[:, :3]
    out[n1:,  3] = np.linalg.norm(second[:, 3:6], axis=1)

fill_mod(dm_mod_11, dm_11_entry_ic, dm_exit_11)
fill_mod(dm_mod_12, dm_12_entry_ic, dm_exit_12)

In [13]:
# Tus datos: distancias y momento dipolar
r12 = dm_mod_11[:,0]
r13 = dm_mod_11[:,1]
r23 = dm_mod_11[:,2]
mu  = dm_mod_11[:,3]   # el módulo del dipolo

# Interpolador RBF
rbf_11 = RBFInterpolator(r12, r13, r23, mu, kernel="cubic", epsilon=1.0)

ValueError: `y` must be a 2-dimensional array.

# Prueba de método de análisis LOO para RBF interpolation